In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, row_number, lit
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("3x2_Backtest").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

dfAll = spark.read.format("csv").option("header", "true").load("combined_games.csv")

dfAll.createOrReplaceTempView("all_shots")

dfAllDates = spark.sql("""select distinct split(game_id, '_')[0] as game_date
from all_shots
order by game_date
""").withColumn("day_number", row_number().over(Window.partitionBy(lit(0)).orderBy(col("game_date")))).select('day_number', 'game_date')

dfAllDates.createOrReplaceTempView("all_dates")

dfAllDates.show(10)

+----------+----------+
|day_number| game_date|
+----------+----------+
|         1|2021-10-19|
|         2|2021-10-20|
|         3|2021-10-21|
|         4|2021-10-22|
|         5|2021-10-23|
|         6|2021-10-24|
|         7|2021-10-25|
|         8|2021-10-26|
|         9|2021-10-27|
|        10|2021-10-28|
+----------+----------+
only showing top 10 rows



In [3]:
# Shot breakdown for first 3 minutes
df3x2_detail = spark.sql("""select game_id, home_team, away_team, relevant_team, shot_type, shot_result, count(*) as total_shots
from all_shots 
where period_type = 'QUARTER' and period = 1 and cast(remaining_seconds_in_period as int) > 540
group by game_id, home_team, away_team, relevant_team, shot_type, shot_result
order by game_id, shot_type""")

df3x2_detail.createOrReplaceTempView("3x2_detail")

df3x2_detail.show()

+--------------+---------+---------+-------------+---------+-----------+-----------+
|       game_id|home_team|away_team|relevant_team|shot_type|shot_result|total_shots|
+--------------+---------+---------+-------------+---------+-----------+-----------+
|2021-10-19_LAL|      LAL|      GSW|          GSW|      2.0|       MAKE|          2|
|2021-10-19_LAL|      LAL|      GSW|          LAL|      2.0|       MISS|          1|
|2021-10-19_LAL|      LAL|      GSW|          GSW|      2.0|       MISS|          2|
|2021-10-19_LAL|      LAL|      GSW|          LAL|      2.0|       MAKE|          2|
|2021-10-19_LAL|      LAL|      GSW|          GSW|      3.0|       MISS|          3|
|2021-10-19_LAL|      LAL|      GSW|          LAL|      3.0|       MISS|          3|
|2021-10-19_LAL|      LAL|      GSW|          GSW|      3.0|       MAKE|          1|
|2021-10-19_MIL|      MIL|      BRK|          MIL|      2.0|       MAKE|          2|
|2021-10-19_MIL|      MIL|      BRK|          MIL|      2.0|     

In [20]:
from pyspark.sql.functions import when

df3x2_makes = spark.sql("""select game_id, home_team, away_team, sum(total_shots) as 3pt_makes
from 3x2_detail 
where shot_result = 'MAKE' and shot_type = 3
group by game_id, home_team, away_team, shot_type, shot_result
order by game_id""")

df3x2_makes.createOrReplaceTempView("3x2_makes")

df3x2_zeros = spark.sql("""select distinct game_id, home_team, away_team, 0 as 3pt_makes
from all_shots where game_id not in (select distinct game_id from 3x2_makes)""")

df3x2_results = df3x2_makes.union(df3x2_zeros).withColumn("3x2_result", when(col("3pt_makes") >= 2, 1).otherwise(0)).sort("game_id")

df3x2_results.createOrReplaceTempView("3x2_results")

df3x2_results.show()

+--------------+---------+---------+---------+----------+
|       game_id|home_team|away_team|3pt_makes|3x2_result|
+--------------+---------+---------+---------+----------+
|2021-10-19_LAL|      LAL|      GSW|        1|         0|
|2021-10-19_MIL|      MIL|      BRK|        1|         0|
|2021-10-20_CHO|      CHO|      IND|        1|         0|
|2021-10-20_DET|      DET|      CHI|        0|         0|
|2021-10-20_MEM|      MEM|      CLE|        1|         0|
|2021-10-20_MIN|      MIN|      HOU|        2|         1|
|2021-10-20_NOP|      NOP|      PHI|        1|         0|
|2021-10-20_NYK|      NYK|      BOS|        3|         1|
|2021-10-20_PHO|      PHO|      DEN|        0|         0|
|2021-10-20_POR|      POR|      SAC|        2|         1|
|2021-10-20_SAS|      SAS|      ORL|        2|         1|
|2021-10-20_TOR|      TOR|      WAS|        2|         1|
|2021-10-20_UTA|      UTA|      OKC|        1|         0|
|2021-10-21_ATL|      ATL|      DAL|        2|         1|
|2021-10-21_GS

In [4]:
# Shot breakdown for first 3 mins by player
df3x2_player_detail = spark.sql("""select game_id, relevant_team, player, shot_type, shot_result, count(*) as total_shots
from all_shots 
where period_type = 'QUARTER' and period = 1 and cast(remaining_seconds_in_period as int) > 540
group by game_id, relevant_team, player, shot_type, shot_result
order by game_id, relevant_team, player, shot_type""")

df3x2_player_detail.createOrReplaceTempView("3x2_player_detail")

df3x2_player_detail.show()

df3x2_player_shot_breakdown = spark.sql("""select d.relevant_team, d.player,
  coalesce((select sum(total_shots) from 3x2_player_detail x where d.player = x.player and d.relevant_team = x.relevant_team and x.shot_type = 3 and x.shot_result = 'MAKE'), 0) as three_pt_made,
  coalesce((select sum(total_shots) from 3x2_player_detail x where d.player = x.player and d.relevant_team = x.relevant_team and x.shot_type = 3), 0) as three_pt_attempts,
  coalesce((select sum(total_shots) from 3x2_player_detail x where d.player = x.player and d.relevant_team = x.relevant_team), 0) as total_shots
from (select distinct relevant_team, player from 3x2_player_detail) d 
order by three_pt_made desc""")

df3x2_player_shot_breakdown.createOrReplaceTempView("3x2_player_shot_breakdown")

df3x2_player_shot_breakdown_pt2 = spark.sql("""select relevant_team, player, three_pt_made, three_pt_attempts, total_shots,
  ROUND((three_pt_made / three_pt_attempts * 100), 2) as 3pt_make_percentage,
  ROUND((three_pt_attempts / total_shots * 100), 2) as 3pt_shot_percentage
from 3x2_player_shot_breakdown""")

df3x2_player_shot_breakdown_pt2.createOrReplaceTempView("3x2_player_shot_breakdown_pt2")

df3x2_player_shot_breakdown_pt2.show()

+--------------+-------------+--------------------+---------+-----------+-----------+
|       game_id|relevant_team|              player|shot_type|shot_result|total_shots|
+--------------+-------------+--------------------+---------+-----------+-----------+
|2021-10-19_LAL|          GSW|      Draymond Green|      2.0|       MISS|          2|
|2021-10-19_LAL|          GSW|        Jordan Poole|      2.0|       MAKE|          1|
|2021-10-19_LAL|          GSW|        Jordan Poole|      3.0|       MISS|          1|
|2021-10-19_LAL|          GSW|        Kevon Looney|      2.0|       MAKE|          1|
|2021-10-19_LAL|          GSW|       Stephen Curry|      3.0|       MISS|          2|
|2021-10-19_LAL|          GSW|       Stephen Curry|      3.0|       MAKE|          1|
|2021-10-19_LAL|          LAL|       Anthony Davis|      3.0|       MISS|          1|
|2021-10-19_LAL|          LAL|      DeAndre Jordan|      2.0|       MAKE|          1|
|2021-10-19_LAL|          LAL|       Kent Bazemore|   

In [5]:
df3x2_total_shots_with_game_number = spark.sql("""SELECT game_id, relevant_team as offense_team, 
case when relevant_team != home_team then home_team else away_team end as defense_team, sum(total_shots) as total_shots
from 3x2_detail
group by game_id, relevant_team, case when relevant_team != home_team then home_team else away_team end
""")

df3x2_total_shots_with_game_number.createOrReplaceTempView("3x2_total_shots")

df3x2_all_shots = spark.sql("""SELECT *, 
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.offense_team = x.relevant_team and shot_type = 3 and shot_result = 'MAKE'), 0) as offense_3_makes,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.offense_team = x.relevant_team and shot_type = 3), 0) as offense_3_shots,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.offense_team = x.relevant_team and shot_type = 2 and shot_result = 'MAKE'), 0) as offense_2_makes,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.offense_team = x.relevant_team and shot_type = 2), 0) as offense_2_shots,

    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.defense_team = x.relevant_team and shot_type = 3 and shot_result = 'MAKE'), 0) as defense_3_makes,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.defense_team = x.relevant_team and shot_type = 3), 0) as defense_3_shots,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.defense_team = x.relevant_team and shot_type = 2 and shot_result = 'MAKE'), 0) as defense_2_makes,
    coalesce((select sum(total_shots) from 3x2_detail x where a.game_id = x.game_id and a.defense_team = x.relevant_team and shot_type = 2), 0) as defense_2_shots
from 3x2_total_shots a
""")
df3x2_all_shots.createOrReplaceTempView("3x2_all_shots")

df3x2_all_stats = spark.sql("""SELECT *, 
    round(offense_3_makes / 3, 2) as offense_3_makes_per_min,
    round(offense_3_shots / 3, 2) as offense_3_shots_per_min,
    round(offense_3_shots / (offense_3_shots + offense_2_shots) * 100) as offense_3_atmpt_pct,

    round(defense_3_makes / 3, 2) as defense_3_makes_per_min,
    round(defense_3_shots / 3, 2) as defense_3_shots_per_min,
    round(defense_3_shots / (defense_3_shots + defense_2_shots) * 100) as defense_3_atmpt_pct
from 3x2_all_shots a
order by game_id
""")
df3x2_all_stats.createOrReplaceTempView("3x2_all_stats")

df3x2_all_stats.show()

# df3x2_rollup_pt1 = spark.sql("""SELECT offense_team as team, round(avg(offense_3_makes), 1) as offense_3_makes, round(avg(offense_3_shots), 1) as offense_3_shots, round(avg(total_shots), 1) as total_shots
# from 3x2_all_stats
# group by offense_team
# """)
# df3x2_rollup_pt1.createOrReplaceTempView("3x2_rollup_pt1")

# df3x2_rollup_pt2 = spark.sql("""select *, 
#     coalesce((select round(avg(offense_3_makes), 1) from 3x2_all_stats x where x.defense_team = a.team group by x.defense_team), 0) as defense_3_makes,
#     coalesce((select round(avg(offense_3_shots), 1) from 3x2_all_stats x where x.defense_team = a.team group by x.defense_team), 0) as defense_3_shots,
#     coalesce((select round(avg(total_shots), 1) from 3x2_all_stats x where x.defense_team = a.team group by x.defense_team), 0) as defense_total_shots
#     from 3x2_rollup_pt1 a

# """)
# df3x2_rollup_pt2.createOrReplaceTempView("3x2_rollup")
# df3x2_rollup_pt2.cache()
# df3x2_rollup_pt2.show()


+--------------+------------+------------+-----------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-------------------+-----------------------+-----------------------+-------------------+
|       game_id|offense_team|defense_team|total_shots|offense_3_makes|offense_3_shots|offense_2_makes|offense_2_shots|defense_3_makes|defense_3_shots|defense_2_makes|defense_2_shots|offense_3_makes_per_min|offense_3_shots_per_min|offense_3_atmpt_pct|defense_3_makes_per_min|defense_3_shots_per_min|defense_3_atmpt_pct|
+--------------+------------+------------+-----------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-----------------------+-----------------------+-------------------+-----------------------+-----------------------+-------------------+
|2021-10-19_LAL|         LAL|         GSW| 

In [27]:
from pyspark.sql.functions import col, round, row_number, lit
from pyspark.sql.window import Window

# Now that we have all the data we need, number each game by offensive team
df3x2_all_stats_with_dates = spark.sql("""select d.game_date, s.*
from 3x2_all_stats s
inner join all_dates d on split(s.game_id, '_')[0] = d.game_date
""").withColumn("game_number", row_number().over(Window.partitionBy(col("offense_team")).orderBy("game_date")))
df3x2_all_stats_with_dates.createOrReplaceTempView("3x2_all_stats_with_dates")
df3x2_all_stats_with_dates.cache()
# df3x2_all_stats_with_dates.show()

dfMatchupsNormal = spark.sql("""select s1.game_number, s1.game_id, s1.offense_team, s1.defense_team,
    round(avg(o5.offense_3_shots), 1) as avg_offense_3_shots,
    round(avg(o5.offense_3_makes), 1) as avg_offense_3_makes,
    round(avg(o5.total_shots), 1) as avg_offense_total_shots,
    round(avg(o5.defense_3_shots), 1) as avg_defense_3_shots,
    round(avg(o5.defense_3_makes), 1) as avg_defense_3_makes,
    count(s1.game_id) as counted_games
from 3x2_all_stats_with_dates s1
inner join 3x2_all_stats_with_dates s2 on s1.game_id = s2.game_id and s1.offense_team = s2.defense_team and s1.defense_team = s2.offense_team
inner join 3x2_all_stats_with_dates o5 on s1.offense_team = o5.offense_team and o5.game_number between s1.game_number - 5 and s1.game_number - 1
group by s1.game_number, s1.game_id, s1.offense_team, s1.defense_team
order by s1.game_id
""")
dfMatchupsNormal.createOrReplaceTempView("matchups_normalized")

# Right now we have 2 rows for each matchup, we need to denormalize it to 1 row
dfMatchups = spark.sql("""select m1.game_id, m1.offense_team, m1.defense_team, m1.avg_offense_3_shots, m1.avg_offense_3_makes, m1.avg_offense_total_shots, m1.avg_defense_3_shots, m1.avg_defense_3_makes,
m2.avg_offense_3_shots as opp_avg_offense_3_shots, m2.avg_offense_3_makes as opp_avg_offense_3_makes, m2.avg_offense_total_shots as opp_avg_offense_total_shots, m2.avg_defense_3_shots as opp_avg_defense_3_shots, m2.avg_defense_3_makes as opp_avg_defense_3_makes,
m1.counted_games, m2.counted_games as opp_counted_games
from matchups_normalized m1
inner join matchups_normalized m2 on m1.game_id = m2.game_id
where m1.offense_team = right(m1.game_id, 3) and m2.defense_team = right(m1.game_id, 3) and m1.counted_games = 5 and m2.counted_games = 5
""")
dfMatchups = dfMatchups.alias("dfMatchups").join(df3x2_results, dfMatchups.game_id == df3x2_results.game_id).select("dfMatchups.*", "3x2_result").orderBy("game_id")
dfMatchups.createOrReplaceTempView("matchups")
dfMatchups.cache()
dfMatchups.show(1000)

+--------------+------------+------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-----------------------+-----------------------+---------------------------+-----------------------+-----------------------+-------------+-----------------+----------+
|       game_id|offense_team|defense_team|avg_offense_3_shots|avg_offense_3_makes|avg_offense_total_shots|avg_defense_3_shots|avg_defense_3_makes|opp_avg_offense_3_shots|opp_avg_offense_3_makes|opp_avg_offense_total_shots|opp_avg_defense_3_shots|opp_avg_defense_3_makes|counted_games|opp_counted_games|3x2_result|
+--------------+------------+------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-----------------------+-----------------------+---------------------------+-----------------------+-----------------------+-------------+-----------------+----------+
|2021-10-29_BRK|         BRK|         IND|                

In [28]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from scipy.stats import binom

# Get 3pt make %
three_pt_make_pct = spark.sql("""select ROUND(sum(three_pt_made) / sum(three_pt_attempts) * 100, 2) as three_pt_make_pct
from 3x2_player_shot_breakdown_pt2""").collect()[0].three_pt_make_pct

print("NBA 3x2 make %: {}%".format(three_pt_make_pct))

@udf(returnType=FloatType()) 
def survivorF(x, y):
    return float(binom.sf(k=1, n=x+y, p=three_pt_make_pct/100))

dfMatchups.withColumn("probYes", survivorF(col("avg_offense_3_shots"), col("opp_avg_offense_3_shots"))).show(200)


NBA 3x2 make %: 37.97%
+--------------+------------+------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-----------------------+-----------------------+---------------------------+-----------------------+-----------------------+-------------+-----------------+----------+----------+
|       game_id|offense_team|defense_team|avg_offense_3_shots|avg_offense_3_makes|avg_offense_total_shots|avg_defense_3_shots|avg_defense_3_makes|opp_avg_offense_3_shots|opp_avg_offense_3_makes|opp_avg_offense_total_shots|opp_avg_defense_3_shots|opp_avg_defense_3_makes|counted_games|opp_counted_games|3x2_result|   probYes|
+--------------+------------+------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-----------------------+-----------------------+---------------------------+-----------------------+-----------------------+-------------+-----------------+----------+----------+
|2